# Ingesting Private Datasets (v1.4)

In [14]:
# imports
import h5py
import specdb

from astropy.table import Table
from linetools import utils as ltu

from specdb.build import privatedb as pbuild


from igmspec.igmspec import IgmSpec

## Test on Single Folder

In [3]:
tree = specdb.__path__[0]+'/build/tests/files'
#os.getenv('DROPBOX_DIR')+'/QSOPairs/data/MMT_redux/'

In [4]:
reload(pbuild)
flux_files = pbuild.grab_files(tree)
len(flux_files)

2

In [5]:
flux_files[:5]

[u'/Users/xavier/local/Python/specdb/specdb/build/tests/files/./SDSSJ001605.89+005654.3_b800_F.fits.gz',
 u'/Users/xavier/local/Python/specdb/specdb/build/tests/files/./SDSSJ001607.27+005653.1_b800_F.fits.gz']

## Directory Tree

In [6]:
tree2 = os.getenv('DROPBOX_DIR')+'/QSOPairs/data/'

In [7]:
reload(pbuild)
mflux_files = pbuild.grab_files(tree2)
len(mflux_files)

22841

In [8]:
mflux_files[:5]

[u'/u/xavier/Dropbox//QSOPairs/data/BOSS/BOSSLyaDR9_cat.fits.gz',
 u'/u/xavier/Dropbox//QSOPairs/data/BOSS/BOSSLyaDR9_cat.fits.gz',
 u'/u/xavier/Dropbox//QSOPairs/data/ESI_redux/SDSSJ001813.89+142455.6_F.fits.gz',
 u'/u/xavier/Dropbox//QSOPairs/data/ESI_redux/SDSSJ002801.18-104933.9_F.fits.gz',
 u'/u/xavier/Dropbox//QSOPairs/data/ESI_redux/SDSSJ002802.61-104936.1_F.fits.gz']

## Meta

In [10]:
igmsp = IgmSpec()

Using /raid/IGMSPEC_DB/IGMspec_DB_v01.hdf5 for the catalog file
Using /raid/IGMSPEC_DB/IGMspec_DB_v01.hdf5 for the DB file
Available surveys: [u'BOSS_DR12', u'GGG', u'HD-LLS_DR1', u'KODIAQ_DR1', u'SDSS_DR7']


In [11]:
ztbl = Table(igmsp.idb.hdf['quasars'].value)

In [12]:
reload(pbuild)
meta = pbuild.mk_meta(flux_files, ztbl, fname=True, skip_badz=True, mdict=dict(INSTR='MMT'))#,debug=True)

Using /raid/IGMSPEC_DB/IGMspec_DB_v01.hdf5 for the catalog file
Using /raid/IGMSPEC_DB/IGMspec_DB_v01.hdf5 for the DB file
Available surveys: [u'BOSS_DR12', u'GGG', u'HD-LLS_DR1', u'KODIAQ_DR1', u'SDSS_DR7']


/Users/xavier/local/Python/specdb/specdb/build/privatedb.py:258: UserWarning: EPOCH not defined.  Filling with 2000.
  warnings.warn("EPOCH not defined.  Filling with 2000.")
/Users/xavier/local/Python/specdb/specdb/build/privatedb.py:267: UserWarning: Meta Column R not defined.  Filling with DUMMY
  warnings.warn("Meta Column {:s} not defined.  Filling with DUMMY".format(clm))
/Users/xavier/local/Python/specdb/specdb/build/privatedb.py:267: UserWarning: Meta Column DATE-OBS not defined.  Filling with DUMMY
  warnings.warn("Meta Column {:s} not defined.  Filling with DUMMY".format(clm))
/Users/xavier/local/Python/specdb/specdb/build/privatedb.py:267: UserWarning: Meta Column SURVEY_ID not defined.  Filling with DUMMY
  warnings.warn("Meta Column {:s} not defined.  Filling with DUMMY".format(clm))
/Users/xavier/local/Python/specdb/specdb/build/privatedb.py:267: UserWarning: Meta Column GRATING not defined.  Filling with DUMMY
  warnings.warn("Meta Column {:s} not defined.  Filling with 

In [13]:
meta[0:3]

sig_zem,flag_zem,flag_survey,zem,RA,DEC,IGM_ID,STYPE,SPEC_FILE,INSTR,EPOCH,R,DATE-OBS,SURVEY_ID,GRATING,TELESCOPE
float64,str9,int64,float64,float64,float64,int64,str3,unicode99,str3,float64,unicode5,unicode8,unicode5,unicode5,unicode5
0.0,BOSS_PCA,1,2.39499998093,4.02454166667,0.948416666667,4635,QSO,/Users/xavier/local/Python/specdb/specdb/build/tests/files/./SDSSJ001605.89+005654.3_b800_F.fits.gz,MMT,2000.0,DUMMY,9999-1-1,DUMMY,DUMMY,DUMMY
0.0,BOSS_PCA,1,2.59719920158,4.03029166667,0.948083333333,4644,QSO,/Users/xavier/local/Python/specdb/specdb/build/tests/files/./SDSSJ001607.27+005653.1_b800_F.fits.gz,MMT,2000.0,DUMMY,9999-1-1,DUMMY,DUMMY,DUMMY


## Spectra

In [14]:
hdf = h5py.File('tmp.hdf5','w')

In [15]:
reload(pbuild)
pbuild.ingest_spectra(hdf, 'test', meta)

Adding test survey to DB


/Users/xavier/local/Python/specdb/specdb/build/utils.py:220: UserWarning: unicode in column SPEC_FILE.  Will convert to str for hdf5
  warnings.warn("unicode in column {:s}.  Will convert to str for hdf5".format(key))
/Users/xavier/local/Python/specdb/specdb/build/utils.py:220: UserWarning: unicode in column R.  Will convert to str for hdf5
  warnings.warn("unicode in column {:s}.  Will convert to str for hdf5".format(key))
/Users/xavier/local/Python/specdb/specdb/build/utils.py:220: UserWarning: unicode in column DATE-OBS.  Will convert to str for hdf5
  warnings.warn("unicode in column {:s}.  Will convert to str for hdf5".format(key))
/Users/xavier/local/Python/specdb/specdb/build/utils.py:220: UserWarning: unicode in column SURVEY_ID.  Will convert to str for hdf5
  warnings.warn("unicode in column {:s}.  Will convert to str for hdf5".format(key))
/Users/xavier/local/Python/specdb/specdb/build/utils.py:220: UserWarning: unicode in column GRATING.  Will convert to str for hdf5
  warn

In [16]:
hdf.close()

## All in One

In [17]:
reload(pbuild)
pbuild.mk_db([tree], ['test'], 'tmp.hdf5', ztbl, fname=True, skip_badz=True, mdict=dict(INSTR='MMT'))

Working on tree: /Users/xavier/local/Python/specdb/specdb/build/tests/files
Using /raid/IGMSPEC_DB/IGMspec_DB_v01.hdf5 for the catalog file
Using /raid/IGMSPEC_DB/IGMspec_DB_v01.hdf5 for the DB file
Available surveys: [u'BOSS_DR12', u'GGG', u'HD-LLS_DR1', u'KODIAQ_DR1', u'SDSS_DR7']
Adding test survey to DB
Wrote tmp.hdf5 DB file


## JSON files for meta table

In [8]:
parse_head = {'DATE-OBS':'DATE', 'TELESCOPE':'TELESCOP','INSTR':'INSTRUME', 'R': True}
mdict = dict(GRATING='ALL', R=8000.)

In [9]:
db_dict = dict(parse_head=parse_head, meta_dict=mdict, maxpix=60000)

In [10]:
jdict = ltu.jsonify(db_dict)

In [12]:
ltu.savejson('tst.json', jdict, easy_to_read=True)

## Running on test suite

In [13]:
igmsp = IgmSpec()

Using /u/xavier/local/Python/igmspec/DB/IGMspec_DB_v02.hdf5 for the catalog file
Using /u/xavier/local/Python/igmspec/DB/IGMspec_DB_v02.hdf5 for the DB file
Available surveys: [u'BOSS_DR12', u'HSTQSO', u'SDSS_DR7', u'KODIAQ_DR1', u'MUSoDLA', u'HD-LLS_DR1', u'2QZ', u'ESI_DLA', u'HDLA100', u'GGG', u'COS-Halos', u'HST_z2', u'COS-Dwarfs', u'XQ-100']
Database is igmspec
Created on 2016-Oct-25


In [15]:
ztbl = Table(igmsp.idb.hdf['quasars'].value)

In [22]:
reload(pbuild)
tree = specdb.__path__[0]+'/data/privateDB'
#pbuild.mk_db.hdf.close()
pbuild.mk_db(tree, 'tst_db.hdf5', ztbl)

IOError: Unable to create file (Unable to truncate a file which is already open)